In [57]:
import os
import string
import re   
import pickle
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
# nltk.download('punkt')

def save_binary(data, file, folder):
    location  = os.path.join(folder, (file+'.pickle') )
    with open(location, 'wb') as ff:
        pickle.dump(data, ff, protocol=pickle.HIGHEST_PROTOCOL)

def load_binary(file, folder):
    location  = os.path.join(folder, (file+'.pickle') )
    with open(location, 'rb') as ff:
        data = pickle.load(ff)
    return data

harakat= load_binary('diacritics','./')

def clear_tashkel(text):
    text = "".join(c for c in text if c not in harakat)
    return text

def get_tashkel(sentence):
    # harakat= load_binary('diacritics','./')
    output = []
    current_haraka = ""
    chIndex = 0
    mode = 0  # mode 0 is character meant (expecting to get character) and mode 1 is tashkeel ment (expecting to get tashkeel)
    while chIndex < (len(sentence)):
        characterTashkeels = []
        if mode == 1:
            while chIndex < (len(sentence)) and sentence[chIndex] in harakat:
                characterTashkeels.append(sentence[chIndex])
                chIndex += 1

            if (len(characterTashkeels) != 0):
                output.append(characterTashkeels)
            else:
                output.append("_") # no tashkeel for now
            # chIndex += 1
            mode = 0
        else:
            mode = 1
            chIndex += 1
    
    if mode == 1: # now I am exepcting tashkeel but the word ended before I find one
        output.append("_")  # _ symbolizes no tashkeel
    return output


arabic_alphabet_set=load_binary('arabic_letters','./')
arabic_alphabet = dict(zip(arabic_alphabet_set, list(range(len(arabic_alphabet_set)))))

def get_char_vector(char):
    if char in arabic_alphabet:
        vector = [0 for _ in range(len(arabic_alphabet))]
        vector[arabic_alphabet[char] ] = 1
        return vector
    else:
        return list(np.ones(len(arabic_alphabet),dtype=int) )


In [58]:
import nltk
import string
import re
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
# read the training set
def readFile(file_path):
    # Open the file for reading
    file_content = None
    with open(file_path, 'r', encoding='utf-8') as file:
        # Read the content of the file
        file_content = file.read()
    return (file_content)

def cleanText(data):
    punctuations=["،","؛","؟",".",",","!",":"," "]
    listTobeChecked = set(tuple(list(harakat)+list(arabic_alphabet.keys())+punctuations))
    for char in data:
        #check if char is not in arabic alphabet or harakat
        if char not in (listTobeChecked):
            data = data.replace(char,'')
    # split text into sentences using regex
    
    data = re.sub('  +',' ', data)
    
    data = re.split(r'[،؛؟.,!:]+', data)
    
    return data
def get_sentences(sentences):
    words_per_sentence = []
    tashkeel_per_sentence = []
    for sentence in sentences:
        sentence = sentence.strip()
        if sentence == '': continue # new line may be followed by full stop
        sentence = word_tokenize(sentence)
        words_per_sentence.append(sentence)
    
    tashkeel_per_sentence = [[None for _ in w ]for w in words_per_sentence]
    for i, x in enumerate(words_per_sentence):
        for j, y in enumerate(x):
            tashkeel = get_tashkel(y)
            words_per_sentence[i][j] = clear_tashkel(y)
            tashkeel_per_sentence[i][j] = tashkeel
    return words_per_sentence, tashkeel_per_sentence

# sent=["عَُمرً"] 

# %%
from gensim.models import  FastText
# Define and train Word2Vec model
def makeModel(dataset):
    model = FastText(sentences=dataset, vector_size=100, window=15, min_count=1, workers=40)
    # Save the trained model (optional)
    model.save("model.model")
    return model

def getCharacterEncoding(word):
    word_embedding = list()
    for w in word:
        word_embedding.append(get_char_vector(w))
    return word_embedding


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, SimpleRNNCell, RNN, Dense,Layer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [59]:
max_sentence_length = 100
embedding_dim = 100
max_char_length = 15
char_vocab_size = 36
num_diacritics = 15
#very small number
lstm_units = 32

In [3]:
# from preprocessing import *

In [4]:
# trainSet = readFile('dataset/train.txt')

In [5]:
# from utils import *
# # from numba import jit, cuda

# # @jit(target_backend='cuda')						 
# def testFunction():
#     print("Hello")
#     sentences_without_diacritics, diacritics = get_sentences(cleanText(trainSet[:5000000]))
#     save_binary(sentences_without_diacritics, 'sentences_without_diacritics_0_1m', './cleaned_data')
#     save_binary(diacritics, 'diacritics_0_1m', './cleaned_data')

# testFunction()

In [6]:
sent=load_binary('sentences_without_diacritics','cleaned_data')


In [7]:
len(sent)

84568

In [8]:
print(sent[0:10])

[['قوله'], ['أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي', 'ابن', 'عرفة'], ['قوله'], ['بلفظ', 'يقتضيه', 'كإنكار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوبه', 'من', 'الدين', 'ضرورة', 'كإلقاء', 'مصحف', 'بقذر', 'وشد', 'زنار', 'ابن', 'عرفة'], ['قول', 'ابن', 'شاس'], ['أو', 'بفعل', 'يتضمنه', 'هو', 'كلبس', 'الزنار', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجاسة', 'والسجود', 'للصنم', 'ونحو', 'ذلك', 'وسحر', 'محمد'], ['قول', 'مالك', 'وأصحابه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك'], ['هو', 'كالزنديق', 'إذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يستتب'], ['قوله', 'لعدم', 'ما', 'تتعلق', 'إلخ', 'أي', 'الوصية', 'قوله', 'ما', 'مر', 'أي', 'قبيل', 'قول', 'المتن', 'لغت', 'ولو', 'اقتصر', 'على', 'أوصيت', 'له', 'بشاة', 'أو', 'أعطوه', 'شاة', 'ولا', 'غنم', 'له', 'عند', 'الموت', 'هل', 'تبطل', 'الوصية', 'أو', 'يشترى', 'له', 'شاة', 'ويؤخذ', 'من', 'قوله', 'الآتي', 'كما', 'لو', 'لم', 'يقل', 'من', 'مالي', 'ولا', 'من', 'غنمي', 'أنها', 'لا', 'تبطل'], ['وعبارة', 'الكنز', 'ولو', '

In [9]:
sentflat= [item for sublist in sent for item in sublist]
print(sentflat[0:10])
model_embedding = makeModel(sent)

['قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي', 'ابن', 'عرفة']


In [10]:
# import fasttext
# from huggingface_hub import hf_hub_download

# model_path = hf_hub_download(repo_id="facebook/fasttext-ar-vectors", filename="model.bin")
# model = fasttext.load_model(model_path)

In [11]:
model_embedding =  FastText.load("model.model")

In [12]:
from gensim.models import Word2Vec, FastText
# load_model = FastText.load_fasttext_format('model.model')
# model=FastText.load('model.model')
# print(model.wv.key_to_index)


# word_embedding = model.get_word_vector('example')
word_embedding = model_embedding.wv['قوله']
print("Embedding for 'example':", word_embedding)


Embedding for 'example': [ 0.57051665 -2.4157765  -4.7408338  -2.4751303  -0.44782665 -0.08962722
 -1.0870793  -0.110075   -2.0900114   2.4319353  -0.7819332   0.8326742
  0.07329872 -0.38973925 -1.478101   -0.9091848  -0.5944232   0.5514373
 -1.0486542  -0.91168594  0.31136057 -0.00751159  1.8379178  -2.148007
 -0.00907751 -0.6861275  -0.2671819   2.0380826  -3.6617682   1.7121551
 -0.6796436   0.7418497   1.0165539  -1.336114    4.170706    2.8560169
  0.01734006  0.50147665  1.9016609   2.771986    2.4589531   1.2556067
  1.0619595  -1.277223    2.0740292   0.71416146 -0.14921214 -0.6483092
  1.3990686  -0.93818355 -2.0018797  -0.7918022  -0.57612467  0.47107184
 -2.2734625  -1.5519607  -1.0733776   1.8456591   1.4949249   1.3811065
  0.9651588   2.0491817   0.11935969  1.764243    0.6018387   1.0847597
  1.0009775   1.046843   -4.0433226   0.28649583 -1.2692508  -0.41779578
  3.2696536  -1.9435546  -0.04343616 -0.25512317  0.00739328  0.10711627
 -2.0240347   0.597299    0.6998953 

In [13]:
def getEmbeddingsSentences(sentences, word2vecmodel):
    embeddingSentences = [] # list of all sentences
    keys = word2vecmodel.wv.key_to_index
    print(keys)
    for s in sentences:
        embeddingTemp = []  # list for one sentence
        for w in s:
            if w in keys:
                embeddingTemp.append(word2vecmodel.wv[w])
            ### unknown OOV till now
        embeddingSentences.append(embeddingTemp)
    return embeddingSentences

embeddingsSentences = getEmbeddingsSentences(sent, model_embedding)


{'في': 0, 'من': 1, 'أو': 2, 'على': 3, 'أن': 4, 'لا': 5, 'قوله': 6, 'أي': 7, 'عليه': 8, 'عن': 9, 'ما': 10, 'لم': 11, 'قال': 12, 'له': 13, 'الله': 14, 'كان': 15, 'بن': 16, 'ذلك': 17, 'وإن': 18, 'ولا': 19, 'به': 20, 'أنه': 21, 'إن': 22, 'ثم': 23, 'فإن': 24, 'إذا': 25, 'لأن': 26, 'ولو': 27, 'لو': 28, 'لأنه': 29, 'وهو': 30, 'فيه': 31, 'إلى': 32, 'كما': 33, 'فلا': 34, 'ابن': 35, 'بعد': 36, 'غير': 37, 'هذا': 38, 'إلا': 39, 'منه': 40, 'هو': 41, 'قبل': 42, 'مع': 43, 'صلى': 44, 'إلخ': 45, 'وسلم': 46, 'كل': 47, 'عنه': 48, 'ولم': 49, 'أبي': 50, 'يكون': 51, 'بأن': 52, 'عبد': 53, 'عند': 54, 'حدثنا': 55, 'حتى': 56, 'بين': 57, 'وقد': 58, 'غيره': 59, 'الذي': 60, 'قول': 61, 'ليس': 62, 'بل': 63, 'وقال': 64, 'شيء': 65, 'ومن': 66, 'لما': 67, 'وفي': 68, 'فإنه': 69, 'فقال': 70, 'قد': 71, 'كانت': 72, 'وإلا': 73, 'بها': 74, 'و': 75, 'بخلاف': 76, 'أيضا': 77, 'إليه': 78, 'يكن': 79, 'أبو': 80, 'فهو': 81, 'فيها': 82, 'وكذا': 83, 'وقوله': 84, 'بما': 85, 'وأما': 86, 'محمد': 87, 'الأول': 88, 'مالك': 89, 'فيما': 90, '

In [14]:
print(embeddingsSentences[0])

[array([ 0.57051665, -2.4157765 , -4.7408338 , -2.4751303 , -0.44782665,
       -0.08962722, -1.0870793 , -0.110075  , -2.0900114 ,  2.4319353 ,
       -0.7819332 ,  0.8326742 ,  0.07329872, -0.38973925, -1.478101  ,
       -0.9091848 , -0.5944232 ,  0.5514373 , -1.0486542 , -0.91168594,
        0.31136057, -0.00751159,  1.8379178 , -2.148007  , -0.00907751,
       -0.6861275 , -0.2671819 ,  2.0380826 , -3.6617682 ,  1.7121551 ,
       -0.6796436 ,  0.7418497 ,  1.0165539 , -1.336114  ,  4.170706  ,
        2.8560169 ,  0.01734006,  0.50147665,  1.9016609 ,  2.771986  ,
        2.4589531 ,  1.2556067 ,  1.0619595 , -1.277223  ,  2.0740292 ,
        0.71416146, -0.14921214, -0.6483092 ,  1.3990686 , -0.93818355,
       -2.0018797 , -0.7918022 , -0.57612467,  0.47107184, -2.2734625 ,
       -1.5519607 , -1.0733776 ,  1.8456591 ,  1.4949249 ,  1.3811065 ,
        0.9651588 ,  2.0491817 ,  0.11935969,  1.764243  ,  0.6018387 ,
        1.0847597 ,  1.0009775 ,  1.046843  , -4.0433226 ,  0.2

In [16]:
save_binary(embeddingsSentences,'embeddings','data')

In [17]:
from utils import *
harakatID   = load_binary('diacritic2id','./')

def get_diacritic_hot_vector(haraka):
    if haraka not in harakatID:
        return list(np.ones(15,dtype=int) )
    vector = [0 for _ in range(len(harakatID))]
    # print("haraka:" + haraka)
    vector[harakatID[haraka] ] = 1
    return vector


In [18]:
diacritics=load_binary('diacritics','cleaned_data')

In [19]:
# harakat   = load_binary('diacritic2id','./')
# for i in harakat:
#     print(i,harakat[i])
#يُسَنُّ أَنْ يُصَانَ عَنْ رَائِحَةٍ كَرِيهَةٍ مِنْ بَصَلٍ وَثُومٍ وَكُرَّاتٍ( 3 / 297 )

corpusDiacList = []
for sentence in diacritics:
    sentenceDiacList = []
    for word in sentence:
        #merge each list 
        oneDiacStr = ''
        diacWordList = []
        for diac in word:
            if diac == '_':
                oneDiacStr = ''
            else:    
                oneDiacStr = ''.join(diac)
            #print(oneDiacStr,harakat[oneDiacStr]) 
            diacWordList.append(get_diacritic_hot_vector(oneDiacStr)) 
        sentenceDiacList.append(diacWordList)       
    corpusDiacList.append(sentenceDiacList)


In [ ]:
# save_binary(corpusDiacList,'labels','data')

In [23]:
embeddingsSentences=load_binary('embeddings','processedd_data')[0:5000]
labels=load_binary('labels','processedd_data')[:5000]
sent=load_binary('sentences_without_diacritics','processedd_data')[:5000]

In [21]:
print(len(embeddingsSentences))

5000


In [24]:
def getInputParameters():
    X_sentence = []  # List for sentence_input
    X_char = []      # List for char_input
    X_scalar = []    # List for scalar_input
    flattened = [item for sublist in labels for item in sublist]
    Y__padded = pad_sequences(flattened, maxlen=max_char_length, padding='post', dtype='float32')
            # List for output

    for i in range(len(embeddingsSentences)):
        for j in range(len(embeddingsSentences[i])):
            # Add embeddings
            X_sentence.append(embeddingsSentences[i])

            # Prepare char_input
            char_encoding = getCharacterEncoding(sent[i][j])
            # Ensure char_encoding is shaped as (max_char_length, char_vocab_size)
            # This might require reshaping or padding depending on your getCharacterEncoding function
            X_char.append(char_encoding)
            # Scalar input
            X_scalar.append([j])

    # Padding X_sentence
    # Assuming max_sentence_length and embedding_dim are defined
    X_sentence_padded = pad_sequences(X_sentence, maxlen=max_sentence_length, padding='post', dtype='float32')
    X_character_padded = pad_sequences(X_char, maxlen=max_char_length, padding='post', dtype='float32')
    # Y__padded = pad_sequences(Y, maxlen=max_char_length, padding='post', dtype='float32')

    # Convert lists to numpy arrays
    X_sentence = np.array(X_sentence_padded)
    X_char = np.array(X_character_padded)
    X_scalar = np.array(X_scalar)

    Y = np.array(Y__padded)

    print(X_sentence.shape, X_char.shape, X_scalar.shape, Y.shape)
    return X_sentence, X_char, X_scalar, Y


X_sentence, X_char, X_scalar, Y = getInputParameters()

(55135, 100, 100) (55135, 15, 36) (55135, 1) (55135, 15, 15)


In [25]:
class SelectHiddenState(Layer):
    def _init_(self, **kwargs):
        super(SelectHiddenState, self)._init_(**kwargs)

    def call(self, lstm_output, scalar_input):
        timestep_index = tf.cast(tf.squeeze(scalar_input, axis=-1), tf.int32)
        selected_state = tf.gather(lstm_output, timestep_index, batch_dims=1, axis=1)
        return selected_state

# Parameters
max_sentence_length = 100
embedding_dim = 100
max_char_length = 15
char_vocab_size = 36
num_diacritics = 15
lstm_units = 32

# Inputs
char_input = Input(shape=(max_char_length, char_vocab_size))
sentence_input = Input(shape=(max_sentence_length, embedding_dim))
scalar_input = Input(shape=(1,), name='scalar_input')

# Padding layer for sentence_input (adjust padding as needed)
# sentence_padding_layer = ZeroPadding1D(padding=(1, 1))  # Example padding
# padded_sentence_input = sentence_padding_layer(sentence_input)
# padding_layer = ZeroPadding1D(padding=(1, 1))  # Example padding
# padded_char_input = padding_layer(char_input)
# BiLSTM layer
bi_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True, return_state=True))
bi_lstm_output, forward_h, forward_c, backward_h, backward_c = bi_lstm(sentence_input)
print(forward_h)
# Select state layer
select_state_layer = SelectHiddenState()
hidden_state_nth_timestep = select_state_layer(bi_lstm_output, scalar_input)

# RNN layer
rnn_cell = SimpleRNNCell(32)
rnn_layer = RNN(rnn_cell, return_sequences=True)
rnn_output = rnn_layer(char_input, initial_state=(forward_c+backward_c)/2)

# Output layer
output_layer = Dense(num_diacritics, activation='softmax')(rnn_output)

# Model
model = Model(inputs=[sentence_input, char_input, scalar_input], outputs=output_layer)
model.compile(optimizer='adam', loss=tf.keras.losses.categorical_crossentropy, metrics=['accuracy'])

# Summary
model.summary()


KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='bidirectional/forward_lstm/PartitionedCall:2', description="created by layer 'bidirectional'")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 100, 100)]           0         []                            
                                                                                                  
 bidirectional (Bidirection  [(None, 100, 64),            34048     ['input_2[0][0]']             
 al)                          (None, 32),                                                         
                              (None, 32),                                                         
                              (None, 32),                                                         
         

In [ ]:
Y.dtype

dtype('float32')

In [26]:
history = model.fit([X_sentence,X_char,X_scalar], Y, epochs= 10)

Epoch 1/10


1723/1723 [==============================] - 92s 49ms/step - loss: 0.4986 - accuracy: 0.8389
Epoch 2/10
1723/1723 [==============================] - 83s 48ms/step - loss: 0.4297 - accuracy: 0.8770
Epoch 3/10
1723/1723 [==============================] - 82s 48ms/step - loss: 0.4111 - accuracy: 0.8804
Epoch 4/10
1723/1723 [==============================] - 91s 53ms/step - loss: 0.3732 - accuracy: 0.8312
Epoch 5/10
1723/1723 [==============================] - 99s 57ms/step - loss: 0.3427 - accuracy: 0.5123
Epoch 6/10
1723/1723 [==============================] - 106s 61ms/step - loss: 0.3260 - accuracy: 0.1902
Epoch 7/10
1723/1723 [==============================] - 86s 50ms/step - loss: 0.3165 - accuracy: 0.1735
Epoch 8/10
1723/1723 [==============================] - 87s 50ms/step - loss: 0.3092 - accuracy: 0.1634
Epoch 9/10
1723/1723 [==============================] - 83s 48ms/step - loss: 0.3038 - accuracy: 0.1702
Epoch 10/10
1723/1723 [==============================] - 88s 

In [27]:
model.evaluate([X_sentence,X_char,X_scalar], Y ,batch_size=3,steps=331)

331/331 [==============================] - 6s 11ms/step - loss: 0.2975 - accuracy: 0.1686


[0.2975306212902069, 0.1685800552368164]

In [36]:
arabic_letters = load_binary('arabic_letters','./')
diacritics2id = load_binary('diacritic2id','./')

# for i in range(len(embeddingsSentences)):
#     for j in range(len(embeddingsSentences[i])):
#         # Add embeddings
#         X_sentence.append(embeddingsSentences[i])
#         char_encoding = getCharacterEncoding(sent[i][j])
#         # Ensure char_encoding is shaped as (max_char_length, char_vocab_size)
#         # This might require reshaping or padding depending on your getCharacterEncoding function
#         # X_char.append(char_encoding)
#         # Scalar input
#         X_scalar.append([j])
# print(arabic_letters)





def myPrediction(testSentences):
    sentences, _ = get_sentences(cleanText(testSentences))
    # sentences.split()



    
    for sentence in sentences:
        X_Embedding_words = []
        X_characters = []
        idsArabic = []
        X_scalar = []
        i = 0
        
        for word in sentence:
            theEmbedding = model_embedding.wv[word]
            # for c in word:
            #     idsArabic.append(i)
            char_encoding = getCharacterEncoding(word) # character encoding of word
            X_Embedding_words.append(theEmbedding)
            X_characters.append(char_encoding)
            X_scalar.append([i])
            i += 1

        X_sentence_padded = pad_sequences(X_Embedding_words, maxlen=max_sentence_length, padding='post', dtype='float32')
        X_character_padded = pad_sequences(X_characters, maxlen=max_char_length, padding='post', dtype='float32')
        # Y__padded = pad_sequences(Y, maxlen=max_char_length, padding='post', dtype='float32')

        # Convert lists to numpy arrays
        X_sentence = np.array(X_sentence_padded)
        X_char = np.array(X_character_padded)
        X_scalar = np.array(X_scalar)
        print(X_sentence.shape, X_char.shape, X_scalar.shape)
        res = model.predict([X_sentence, X_char, X_scalar])
        print(res)

myPrediction('يُسَنُّ أَنْ يُصَانَ عَنْ رَائِحَةٍ كَرِيهَةٍ مِنْ بَصَلٍ وَثُومٍ وَكُرَّاتٍ( 3 / 297 )')

(10, 100) (10, 15, 36) (10, 1)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [55]:
import csv

def writeToCSV(data):
    with open('submission.csv', mode='w', newline='') as csv_file:  # Added newline=''
        fieldnames = ['ID', 'label']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for i in range(len(data)):
            writer.writerow({'ID': i, 'label': data[i]})

In [56]:
from utils import *
from preprocessing import *
from gensim.models import Word2Vec, FastText
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

max_sentence_length = 30
embedding_dim = 100
max_char_length = 15
char_vocab_size = 36
num_diacritics = 15
model1h5=load_model('my_model1.h5')


def split_lists(input_list, max_length):
    result_lists = []
    for inner_list in input_list:
        for i in range(0, len(inner_list), max_length):
            result_lists.append(inner_list[i:i + max_length])
    return result_lists


def test_sent(path):
    testset = readFile(path)
    sent=cleanText(testset)
    
    sent, _ = get_sentences(sent)
    sent=split_lists(sent,max_sentence_length)

    embeddding_model=FastText.load('model.model')

    def getEmbeddings(sentences, word2vecmodel):
        embeddingSentences = [] # list of all sentences
        keys = word2vecmodel.wv.key_to_index
        for s in sentences:
            embeddingTemp = []  # list for one sentence
            for w in s:
                embeddingTemp.append(word2vecmodel.wv[w])
            embeddingSentences.append(embeddingTemp)
        return embeddingSentences

    embeddingsSentences = getEmbeddings(sent, embeddding_model)

    X_sentence = []  # List for sentence_input
    X_char = []      # List for char_input
    X_scalar = []    # List for scalar_input

    
    for i in range(len(embeddingsSentences)):
        for j in range(len(embeddingsSentences[i])):
            for char in sent[i][j]:
                X_sentence.append(embeddingsSentences[i])
                X_char.append(get_char_vector(char))
                X_scalar.append([j])


    X_sentence_padded = pad_sequences(X_sentence, maxlen=max_sentence_length, padding='post', dtype='float32')
    # Convert lists to numpy arrays
    X_sentence = np.array(X_sentence_padded)
    X_char = np.array(X_char)
    X_scalar = np.array(X_scalar)

    return X_sentence, X_char, X_scalar
X_sentence, X_char, X_scalar=test_sent("Delivery\sample_testset_with_submission\sample_test_no_diacritics.txt")
predictions=model1h5.predict([X_sentence,X_char,X_scalar])
pred=np.argmax(predictions, axis=-1)
print()
writeToCSV(pred)
for i in range(len(pred)):
    print(i,pred[i])

6/6 [==============================] - 2s 28ms/step

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 6
10 4
11 0
12 0
13 6
14 0
15 4
16 6
17 0
18 6
19 0
20 4
21 4
22 0
23 0
24 0
25 0
26 6
27 4
28 0
29 0
30 0
31 6
32 4
33 0
34 0
35 0
36 6
37 0
38 0
39 4
40 0
41 0
42 4
43 0
44 0
45 0
46 0
47 6
48 4
49 0
50 6
51 4
52 0
53 0
54 0
55 0
56 6
57 14
58 0
59 0
60 0
61 4
62 6
63 0
64 4
65 0
66 0
67 0
68 6
69 4
70 0
71 0
72 0
73 6
74 0
75 0
76 0
77 4
78 0
79 0
80 0
81 14
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 6
93 0
94 0
95 4
96 6
97 0
98 4
99 0
100 0
101 0
102 14
103 0
104 0
105 4
106 0
107 0
108 6
109 0
110 0
111 0
112 0
113 0
114 4
115 6
116 0
117 4
118 0
119 0
120 0
121 14
122 0
123 0
124 4
125 6
126 0
127 0
128 14
129 6
130 0
131 4
132 14
133 2
134 4
135 0
136 4
137 0
138 0
139 0
140 4
141 6
142 0
143 0
144 4
145 0
146 0
147 4
148 0
149 2
150 4
151 0
152 4
153 6
154 0
155 0
156 0
157 0
158 4
159 0
160 14
161 0
162 14
163 4
164 0
165 0
166 0
167 0
168 0
169 2
170 4
171 0
172 14
173 0
174 

In [ ]:
import tensorflow as tf
with tf.device('/device:CPU:0'):

# Create two random matrices

    a = tf.random.normal([1000, 1000])

    b = tf.random.normal([1000, 1000])

    # Multiply the matrices

    c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[ 12.199187  -18.303337    1.4164754 ... -45.291946  -43.909237
   11.5318775]
 [-15.765798  -14.711167  -17.025244  ...  21.670637   23.795555
  -10.237402 ]
 [ 26.30426   -12.242771    3.1339736 ...   2.6476212 -16.818653
   19.0809   ]
 ...
 [ 31.12152    15.558274  -11.423328  ...   1.6794755   0.3871634
  -49.38184  ]
 [-11.694465  -11.766845   47.311592  ... -41.749184  -48.630306
   17.7858   ]
 [-42.82271   -15.562574   41.811913  ...  36.006863   22.38664
   36.572754 ]], shape=(1000, 1000), dtype=float32)
